In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from tqdm import tqdm

In [2]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [85]:
# 爬取每个具体网站的id号
webid=[]
for i in range(5):
    url = f"https://arxiv.org/list/cs/pastweek?skip={(i-1)*25}&show={25*i}"
    web = requests.get(url=url,headers=headers)
    soup = BeautifulSoup(web.content,"html.parser")

    tmp = soup.find_all("dt")
    webid = webid+[ele.select_one("span a").get_text().replace("arXiv:","") for ele in tmp]

In [87]:
# 爬取每个网站上面的具体信息
Title=[]
Subjects=[]
Authors=[]
Abstracts=[]

for m in tqdm(webid, desc='Scraping ArXiv'):
    url= f"https://arxiv.org/abs/{m}"
    web = requests.get(url=url,headers=headers)
    soup = BeautifulSoup(web.content,"html.parser")

    abstract = soup.select_one("blockquote").get_text().strip().replace("Abstract:","")
    subject = soup.select("div h1")[0].getText().replace("Computer Science > ","")
    title = soup.select("div h1")[1].get_text().replace("Title:","")
    authors=[tmp.get_text() for tmp in soup.select("div.authors a")]

    Title.append(title)
    Subjects.append(subject)
    Authors.append(authors)
    Abstracts.append(abstract)

In [88]:
# 转化成为dataframe
data = {
    'Title': Title,
    'Subjects': Subjects,
    'Authors': Authors,
    'Abstracts': Abstracts
}

# 创建 DataFrame
df = pd.DataFrame(data)
df.head()

,Title,Subjects,Authors,Abstracts
0,Point Could Mamba: Point Cloud Learning via St...,Computer Vision and Pattern Recognition,"[Tao Zhang, Xiangtai Li, Haobo Yuan, Shunping ...","In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware P...,Computation and Language,"[Qingyan Guo, Rui Wang, Junliang Guo, Xu Tan, ...",While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video Str...,Multimedia,"[Imran Khan, Tuyen X. Tran, Matti Hiltunen, Th...",Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for loc...,Machine Learning,"[János Kramár, Tom Lieberum, Rohin Shah, Neel ...",Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pre...,"Distributed, Parallel, and Cluster Computing","[Joshua Dennis Booth, Hongyang Sun, Trevor Gar...",The solution of a sparse system of linear equa...


In [89]:
#存入csv
df.to_csv("./CS.csv")